# Descarga de datos de hospitalizados desde Sanidad

## Objetivo

Vamos a hacer una lectura de los datos de hospitalizados publicados en los informes diarios de datos de Covid19 publicados por Sanidad. Este es un [ejemplo](https://www.mscbs.gob.es/profesionales/saludPublica/ccayes/alertasActual/nCov/documentos/Actualizacion_265_COVID-19.pdf) en página 3,  publicado del 4 de Diciembre 2020.
  
Manuel H. Arias  
[@walyt](https://twitter.com/walyt)  

[#escovid19data](https://github.com/montera34/escovid19data)

versión definitiva documentada, para ser publicar en el repo

[@walyt](https://twitter.com/walyt)

## Código

Tenemos un montón de librerias con las que vamos a trabajar, no he tenido ningún problema en instalar aquellas no disponibles en el entorno Anaconda con el que trabajo por medio de `pip install libreria` realizado desde un terminal abierto desde el entorno `env`.

In [15]:
import os.path as pth
import datetime as dt
import time
from glob import glob
import re
import pandas as pd
import numpy as np

import requests
from shutil import copyfile

import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter
from matplotlib import cm
import matplotlib.dates as mdates
import matplotlib.ticker as ticker
from matplotlib.dates import (YEARLY, MONTHLY, DateFormatter, WeekdayLocator, MonthLocator,DayLocator,
                              rrulewrapper, RRuleLocator, drange)
import seaborn as sns
import matplotlib.colors as colors

import numpy as np
from datetime import datetime,timedelta
import seaborn as sns
%matplotlib inline

import fitz
# https://pypi.org/project/PyMuPDF/#description

Preparamos las expresiones regulares que nos ayudarán en la interpretación de la información que sacamos de los pdf.

Definimos variables que nos ayuden en la gestión de los nombres de los ficheros.

In [16]:
fuente_ficheros_pdf_sanidad = '/Users/mharias/documents/proyectos/covid/sanidad_fallecidos/pdfs_sanidad/'
dir_csv = '/Users/mharias/documents/proyectos/covid/sanidad_hospital/data/'
URL_reg='https://www.mscbs.gob.es/profesionales/saludPublica/ccayes/alertasActual/nCov/documentos/Actualizacion_{:02d}_COVID-19.pdf'

## Descarga de los ficheros de Sanidad

### Función de para descargar un fichero pdf, copiada del script de [@alfonsotwr](https://github.com/alfonsotwr/snippets/tree/master/covidia-cam)

In [17]:
def descarga(url,num):
    print('Descargando:', url)
    fn=fuente_ficheros_pdf_sanidad+str(num)+'.pdf'
    headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}
    with requests.Session() as s:
        r = s.get(url, headers=headers)
    if r.status_code == requests.codes.ok:
        with open(fn, 'wb') as fp:
                fp.write(r.content)
    else:
        print ('Error con el ',num)
    return True

### Descarga de un rango o de un solo pdf

Descargamos el rango completo en el caso de que sea la primera vez. Arrancamos con el 77, pues no pude descifrar el formato de los pdf anteriores.

In [18]:
#for i in range(287,297):
#    descarga(URL_reg.format(i),i)

### Descarga de un solo fichero

Como ejemplo 266 corresponde al Viernes 4 Diciembre 2020, grabamos el fichero en el directorio local con el nº de orden del documento.

In [19]:
n_fichero = 347

In [20]:
descarga(URL_reg.format(n_fichero),n_fichero)

Descargando: https://www.mscbs.gob.es/profesionales/saludPublica/ccayes/alertasActual/nCov/documentos/Actualizacion_347_COVID-19.pdf


True

## Parseado y preparación del pandas

In [21]:
columnas = ['fecha','ccaa','Hosp_Totales','Hosp_7días','UCI_Totales','UCI_7dias','Hosp_actual','%CamasCovid',
        'UCI_actual','%CamasUCI','Ingresos_24horas','Altas_24horas']
datos = pd.DataFrame(columns=columnas)

fichero auxiliar que mapea nº de informes con fechas....Mejora: automatizarlo

In [22]:
claves={}
indice=76
rango= np.arange(datetime(2020,4,16),datetime.today(),timedelta(days=1)).astype(datetime)
for day in rango:
    if ((day==datetime(2021,4,3)) | (not (((day>datetime(2020,6,30))&(day.weekday() in [5,6])) |\
            (day==datetime(2020,12,6)) | (day==datetime(2020,12,8)) | (day==datetime(2020,12,25)) | \
           (day==datetime(2021,1,1)) | (day==datetime(2021,1,6)) | (day==datetime(2021,3,19))|(day==datetime(2021,4,2))))):
        print (day,indice+1)
        indice+=1
        claves[indice]=day.strftime('%-d/%-m/%Y')

2020-04-16 00:00:00 77
2020-04-17 00:00:00 78
2020-04-18 00:00:00 79
2020-04-19 00:00:00 80
2020-04-20 00:00:00 81
2020-04-21 00:00:00 82
2020-04-22 00:00:00 83
2020-04-23 00:00:00 84
2020-04-24 00:00:00 85
2020-04-25 00:00:00 86
2020-04-26 00:00:00 87
2020-04-27 00:00:00 88
2020-04-28 00:00:00 89
2020-04-29 00:00:00 90
2020-04-30 00:00:00 91
2020-05-01 00:00:00 92
2020-05-02 00:00:00 93
2020-05-03 00:00:00 94
2020-05-04 00:00:00 95
2020-05-05 00:00:00 96
2020-05-06 00:00:00 97
2020-05-07 00:00:00 98
2020-05-08 00:00:00 99
2020-05-09 00:00:00 100
2020-05-10 00:00:00 101
2020-05-11 00:00:00 102
2020-05-12 00:00:00 103
2020-05-13 00:00:00 104
2020-05-14 00:00:00 105
2020-05-15 00:00:00 106
2020-05-16 00:00:00 107
2020-05-17 00:00:00 108
2020-05-18 00:00:00 109
2020-05-19 00:00:00 110
2020-05-20 00:00:00 111
2020-05-21 00:00:00 112
2020-05-22 00:00:00 113
2020-05-23 00:00:00 114
2020-05-24 00:00:00 115
2020-05-25 00:00:00 116
2020-05-26 00:00:00 117
2020-05-27 00:00:00 118
2020-05-28 00:0

In [23]:
comunidades=['Andalucía','Aragón','Asturias','Baleares','Canarias','Cantabria','Castilla La Mancha',
             'Castilla y León','Cataluña','Ceuta','C. Valenciana','Extremadura','Galicia',
             'Madrid','Melilla','Murcia','Navarra','País Vasco','La Rioja']

# antes 235

In [24]:
rango_informes = np.concatenate([np.arange(88,137),np.arange(138,234)])
#rango_informes = np.concatenate([np.arange(88,90)])
opt='text'
for informe in rango_informes:
    fn=fuente_ficheros_pdf_sanidad+str(informe)+'.pdf'
    doc = fitz.open(fn)
    page = doc.load_page(1)
    text = page.get_text(opt)
    
    for comunidad in comunidades:
        regexp = r'{}[*]* '.format(comunidad)+'((\n\d*[,.]?\d+¥? ){6})'
        regexp = r'{}[*]* '.format(comunidad)+'(((\n\d*[,.]?\d+¥? )|(\n- )){6})'
        regexp = r'{}[*¥]* '.format(comunidad)+'(((\n\d*[,.]?\d+¥? )|(\n- )|(\n )){6})'
        cadena = re.search(regexp,text).group(1).replace('¥','').split('\n')[1:]
        # c1,c2,c3,c4,c5,c6 = re.search(regexp,text).group(1).replace('\n','').split()
                                                                       
        cadena = [i.rstrip().lstrip().replace('.','') if i not in ['',' '] else 0 for i in cadena]
        
        #c1,c2,c3,c4,c5,c6 = re.search(regexp,text).group(1).split('\n')[1:]
        c1,c2,c3,c4,c5,c6 = cadena
        datos = datos.append({'fecha':claves[informe],
                                                 'ccaa':comunidad,'Hosp_Totales':c1,
                                               'Hosp_7días':c2,'UCI_Totales':c3,
                                                 'UCI_7dias':c4},ignore_index=True)
        
        
        #print (informe,'--->',comunidad,c1,'-',c2,'-',c3,'-',c4,'-',c5,'-',c6)
        #print (cadena)

In [25]:
datos.head(20)

,fecha,ccaa,Hosp_Totales,Hosp_7días,UCI_Totales,UCI_7dias,Hosp_actual,%CamasCovid,UCI_actual,%CamasUCI,Ingresos_24horas,Altas_24horas
0,27/4/2020,Andalucía,5768,20,720,4,NaN,NaN,NaN,NaN,NaN,NaN
1,27/4/2020,Aragón,2403,8,257,0,NaN,NaN,NaN,NaN,NaN,NaN
2,27/4/2020,Asturias,1814,24,137,3,NaN,NaN,NaN,NaN,NaN,NaN
3,27/4/2020,Baleares,1062,0,166,0,NaN,NaN,NaN,NaN,NaN,NaN
4,27/4/2020,Canarias,881,1,171,0,NaN,NaN,NaN,NaN,NaN,NaN
5,27/4/2020,Cantabria,995,4,78,0,NaN,NaN,NaN,NaN,NaN,NaN
6,27/4/2020,Castilla La Mancha,8444,27,579,14,NaN,NaN,NaN,NaN,NaN,NaN
7,27/4/2020,Castilla y León,7703,50,518,3,NaN,NaN,NaN,NaN,NaN,NaN
8,27/4/2020,Cataluña,25799,134,2724,141,NaN,NaN,NaN,NaN,NaN,NaN
9,27/4/2020,Ceuta,10,0,4,0,NaN,NaN,NaN,NaN,NaN,NaN


## desde 235

In [26]:
rango = np.concatenate([np.arange(235,266),np.arange(267,n_fichero+1)])
opt='text'
for i in rango:
    fn=fuente_ficheros_pdf_sanidad+str(i)+'.pdf'
    doc = fitz.open(fn)
    numero_pagina = 3 if i==282 else 2
    page = doc.load_page(numero_pagina)
    text = page.get_text(opt)
    
    for comunidad in comunidades:
        regexp = r'{} '.format(comunidad)+'((\n\d*[,.]?\d+¥? ){4})'+\
        '(\n \n(\w+[ *.\w]*) )((\n\d*[,.]?\d+¥? ))'+\
        '((\n+\d{1,2}[,]?\d*% ))'+\
        '((\n\d*[,.]?\d+¥? ))'+\
        '((\n+\d{1,2}[,]?\d*% ))'+\
        '(((\n\d*[,.]?\d+¥? )){2})'
        pattern = re.compile(regexp)
        c1,c2,c3,c4 = re.search(pattern,text).group(1).replace('¥','').replace('.','').replace('\n','').split()
        c5 = re.search(pattern,text).group(6).replace('¥','').replace('.','').replace('\n','')
        c6 = re.search(pattern,text).group(8).replace('¥','').replace('.','').replace('\n','')
        c7 = re.search(pattern,text).group(9).replace('¥','').replace('.','').replace('\n','')
        c8 = re.search(pattern,text).group(11).replace('¥','').replace('.','').replace('\n','')
        c9, c10 = re.search(pattern,text).group(13).replace('¥','').replace('.','').replace('\n','').split()
        
        #print (i,'--->',comunidad,c1,'-',c2,'-',c3,'-',c4,'-',c5,'-',c6,'-',c7,'-',c8,'-',c9,'-',c10)
        datos = datos.append({'fecha':claves[i],
                                                 'ccaa':comunidad,'Hosp_Totales':c1,
                                               'Hosp_7días':c2,'UCI_Totales':c3,
                                                 'UCI_7dias':c4,'Hosp_actual':c5,
                                                 '%CamasCovid':c6,'UCI_actual':c7,
                                                 '%CamasUCI':c8,'Ingresos_24horas':c9,
                                                 'Altas_24horas':c10},ignore_index=True)
        

In [27]:
datos.tail(20)

,fecha,ccaa,Hosp_Totales,Hosp_7días,UCI_Totales,UCI_7dias,Hosp_actual,%CamasCovid,UCI_actual,%CamasUCI,Ingresos_24horas,Altas_24horas
4863,5/4/2021,La Rioja,3224,14,364,1,60,"8,30%",19,"35,85%",12,3
4864,6/4/2021,Andalucía,41502,400,4313,39,1367,"7,55%",287,"15,14%",199,159
4865,6/4/2021,Aragón,13000,149,1212,13,308,"7,35%",47,"20,26%",32,46
4866,6/4/2021,Asturias,7989,101,605,9,256,"7,32%",68,"20,86%",37,19
4867,6/4/2021,Baleares,4497,9,670,0,70,"2,17%",20,"7,19%",8,2
4868,6/4/2021,Canarias,4785,95,958,8,290,"5,29%",86,"18,03%",33,31
4869,6/4/2021,Cantabria,2823,50,390,10,78,"4,96%",21,"17,50%",12,15
4870,6/4/2021,Castilla La Mancha,18507,26,1353,4,323,"6,65%",58,"15,38%",30,29
4871,6/4/2021,Castilla y León,25823,178,2044,11,517,"7,64%",123,"22,82%",72,36
4872,6/4/2021,Cataluña,38708,141,3195,11,2340,"9,42%",532,"37,00%",236,129


In [28]:
datos.to_csv(dir_csv+'datos_sanidad_hosp.csv',index=False,index_label=False)

# Taller de reparación

In [15]:
rango = np.arange(326,327)
for i in rango:
    fn=fuente_ficheros_pdf_sanidad+str(i)+'.pdf'
    doc = fitz.open(fn)
    numero_pagina = 3 if i==282 else 2
    page = doc.load_page(numero_pagina)
    text = page.get_text(opt)
    
    for comunidad in comunidades:
        regexp = r'{}[¥\*]* '.format(comunidad)+'((\n\d*[,.]?\d+¥? ){4})'+\
        '(\n \n(\w+[ *.\w]*) )((\n\d*[,.]?\d+¥? ))'+\
        '((\n+\d{1,2}[,]?\d*% ))'+\
        '((\n\d*[,.]?\d+¥? ))'+\
        '((\n+\d{1,2}[,]?\d*% ))'+\
        '(((\n\d*[,.]?\d+¥? )){2})'
        pattern = re.compile(regexp)
        print (comunidad)
        c1,c2,c3,c4 = re.search(pattern,text).group(1).replace('¥','').replace('.','').replace('\n','').split()
        c5 = re.search(pattern,text).group(6).replace('¥','').replace('.','').replace('\n','')
        c6 = re.search(pattern,text).group(8).replace('¥','').replace('.','').replace('\n','')
        c7 = re.search(pattern,text).group(9).replace('¥','').replace('.','').replace('\n','')
        c8 = re.search(pattern,text).group(11).replace('¥','').replace('.','').replace('\n','')
        c9, c10 = re.search(pattern,text).group(13).replace('¥','').replace('.','').replace('\n','').split()
        
        #print (i,'--->',comunidad,c1,'-',c2,'-',c3,'-',c4,'-',c5,'-',c6,'-',c7,'-',c8,'-',c9,'-',c10)
        datos = datos.append({'fecha':claves[i],
                                                 'ccaa':comunidad,'Hosp_Totales':c1,
                                               'Hosp_7días':c2,'UCI_Totales':c3,
                                                 'UCI_7dias':c4,'Hosp_actual':c5,
                                                 '%CamasCovid':c6,'UCI_actual':c7,
                                                 '%CamasUCI':c8,'Ingresos_24horas':c9,
                                                 'Altas_24horas':c10},ignore_index=True)
        

Andalucía
Aragón
Asturias
Baleares
Canarias
Cantabria
Castilla La Mancha
Castilla y León
Cataluña
Ceuta
C. Valenciana
Extremadura
Galicia
Madrid
Melilla
Murcia
Navarra
País Vasco
La Rioja


In [16]:
datos

,fecha,ccaa,Hosp_Totales,Hosp_7días,UCI_Totales,UCI_7dias,Hosp_actual,%CamasCovid,UCI_actual,%CamasUCI,Ingresos_24horas,Altas_24horas
0,27/4/2020,Andalucía,5768,20,720,4,NaN,NaN,NaN,NaN,NaN,NaN
1,27/4/2020,Aragón,2403,8,257,0,NaN,NaN,NaN,NaN,NaN,NaN
2,27/4/2020,Asturias,1814,24,137,3,NaN,NaN,NaN,NaN,NaN,NaN
3,27/4/2020,Baleares,1062,0,166,0,NaN,NaN,NaN,NaN,NaN,NaN
4,27/4/2020,Canarias,881,1,171,0,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
4802,5/3/2021,Melilla,504,14,76,2,18,"9,89%",3,"17,65%",4,1
4803,5/3/2021,Murcia,8565,67,1101,11,172,"4,16%",66,"13,58%",14,19
4804,5/3/2021,Navarra,5152,27,451,5,70,"3,61%",14,"10,69%",4,4
4805,5/3/2021,País Vasco,12718,54,1074,8,494,"9,88%",110,"25,11%",65,72


In [17]:
text

' \n \n Centro de Coordinación de Alertas \ny Emergencias Sanitarias \n \nPágina 3 de 21 \n \nSECRETARÍA DE ESTADO \nDE SANIDAD \n \nDIRECCIÓN GENERAL DE \nSALUD PÚBLICA  \n \n \n \nTabla 2. Casos de COVID-19 que han precisado hospitalización e ingreso en UCI. \nTabla 3. Situación capacidad asistencial y actividad Covid-19 en  hospitales  \n \nCasos que han precisado \nhospitalización \nCasos que han ingresado en \nUCI \n \nCCAA \nTotal Pacientes \nCOVID \nhospitalizados \n% Camas \nOcupadas \nCOVID \nNúmero \nPacientes \nCOVID en \nUCI \n% Camas \nOcupadas \nUCI \nCOVID \nIngresos \nCOVID \núltimas \n24h \nAltas \nCOVID \núltimas \n24h \nCCAA \nTotal \nCon fecha de \ningreso en los \núltimos 7 días \nTotal \nCon fecha de \ningreso en UCI en \nlos últimos 7 días \n \nAndalucía \n38.413 \n333 \n3.857 \n20 \n \nAndalucía \n1.503 \n8,33% \n351 \n18,71% \n150 \n202 \nAragón \n12.391 \n154 \n1.126 \n11 \n \nAragón \n313 \n7,45% \n69 \n28,99% \n41 \n45 \nAsturias \n7.416 \n136 \n559 \n15 \n 